# SQL

Le SQL (Structured Query Lnaguage) est une norme définie depuis 1986.

Le modèle de données est la table de données ou si vous préférez la structure de données.

## SGBD

Nous allons travailler avec MySQL ou MariaDB et surtout avec le modèle relationnel InnoDB.

Il existe plusieurs moteurs de base de données, pour MySQL vous avez : MyISAM, Innodb, Archive, Memory par exemple.

**MyISAM** est un moteur qui ne supporte pas les transactions, mais possède des fonctionnalités de recherche de texte.

**InnoDB** est le moteur qui est le plus utilisé. Il supporte le mode transactionnel et les contraintes référentielles (clés étrangères).

**Memory** est un moteur de stockage dans la mémoire vive : RAM de l'ordinateur. On peut l'utiliser par exemple pour des systèmes de cache pour une application.

## Modèle de données

### Table de données

Le modèle de données relationnel repose sur des principes théoriques rigoureux (algèbre relationnel). La **table relationnelle** est la structure de données qui contient des lignes (rows) et des colonnes (columns) qui décrivent les enregistrements.

### Les clés

La clé primaire (Primary Key) d'une table est l'ensemble minimal de colonnes qui permet d'identifier **de manière unique** chaque enregistrement. Dans une table il faut trouver les clés candidates pouvant remplir ce rôle et sinon utiliser un identifiant numérique de type entier auto-incrémenté.

Attention, il ne peut y avoir qu'une seule clé primaire dans une table relationnel.

La clé étrangère (Foreign Key), elle référence dans la majorité des cas une clé primaire d'une autre table. Il peut y avoir plusieurs clés étrangères dans une même table.

## Base de données

C'est un regroupement logique d'objets comme les tables, index, vues, procédures, ... Stockés sur le disque dur. MySQL définit la notion d'hôte (host) c'est la machine qui héberge le SGDB.

Clairement votre base de données hébergera vos tables, l'accès à votre base de données sera également conditionné par un login/password et une adresse ou host.

Pour se connecter avec votre mot de passe à la base de données books :

```bash
mysql -u root -p
use book
```

Plus rapidement vous pouvez également taper la ligne suivante dans votre terminale :

```bash
mysql -u root -p --database book
```

Pour quitter la base de données :

```bash
mysql> quit
```

Pour afficher les commandes disponibles dans MySQL tapez une fois sur le serveur la commande suivante :

```bash
mysql>?
```

**Memory** possède le stockage des données et index en RAM.

## LMD langage de manipulation 

Pour manipuler les donner SQL propose trois instructions :

- Insertion d'enregistrement : INSERT

- Modification de données : UPDATE

- Suppression d'enregistrement : DELETE et TRUNCATE

Pour les besoins du cours nous allons créer des tables SQL, on vous donne le code de ces tables dans le cours. Comme nous utilisons le modèle relationnel, nous utilisons des clés secondaires (FK) pour mettre en relation les structures de données (tables relationnelles).

Dans un premier temps créer la base de données et les tables dans cette base.

## Création de la base de données & des tables

```sql
CREATE DATABASE blog DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
```
Pour la supprimer vous n'aurez qu'à taper la ligne :

```sql
drop database blog
```

### Définition des tables

Table des utilisateurs users, notez les cotes couchées échappent les caractères dans MySQL :

```sql

CREATE TABLE `users` (
    `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
    `email` VARCHAR(100),
    `password` VARCHAR(100),
    `status` ENUM('administrator', 'visitor', 'author')
    NOT NULL DEFAULT 'visitor',
    PRIMARY KEY (`id`),
    CONSTRAINT `un_email` UNIQUE (`email`)
  ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

```

Table des posts, un post est écrit par au plus un auteur et un auteur peut avoir écrit plusieurs posts.

```sql
CREATE TABLE posts (
    `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
    `title` VARCHAR(100),
    `thematic` CHAR(5) DEFAULT NULL, -- 5 octets donc 5 caractères
    `lesson` VARCHAR(30) DEFAULT NULL, -- 30 octets le nom des leçons
    `user_id` INT UNSIGNED DEFAULT NULL,
    `content` TEXT,
    `status` ENUM('published', 'unpublished') NOT NULL DEFAULT 'unpublished',
    published_at DATETIME NOT NULL,
    PRIMARY KEY (id),
    CONSTRAINT `posts_users_id_foreign` FOREIGN KEY (`user_id`) REFERENCES users (`id`) ON DELETE SET NULL
  ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

  ```

  Table des catégories, une catégorie peut avoir de 0 à N post(s) et un post peut être associé à 0 à N catégorie, dans ce cas la relation est dite N:N :

  ```sql
CREATE TABLE categories (
    `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
    `title` VARCHAR(100),
    PRIMARY KEY (`id`)
  ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

/* Table de liaison */
CREATE TABLE category_post (
  `post_id` INT UNSIGNED,
  `category_id` INT UNSIGNED,
  CONSTRAINT `category_post_posts_post_id_foreign` FOREIGN KEY(`post_id`) REFERENCES posts(`id`) ON DELETE CASCADE,
  CONSTRAINT `category_post_categories_post_id_foreign` FOREIGN KEY(`category_id`) REFERENCES categories(`id`) ON DELETE CASCADE,
  CONSTRAINT `un_post_id_category_id` UNIQUE KEY (`post_id`, `category_id` )
  ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
  ```

Nous allons maintenant insérer des données de manière **multiple** depuis la console :

```sql

-- enregistrer des lignes dans la table (structure données)

INSERT INTO `users` 
-- champs 
  (`email`,`password`, `status`)
VALUES
-- les données qui correspondent aux noms des champs
  ('tony@tony.fr', '123', 'administrator'),
  ('alan@alan.fr', '123', 'author');
```

Insertion des posts, recherchez l'identifiant de votre auteur dans la table users puis. On utilise NOW qui est une fonction de MySQL et qui retourne le datetime actuel, pour la tester dans MySQL tapez la ligne de commande suivante :

```sql

mysql> SELECT NOW();
/*
+---------------------+
| NOW()               |
+---------------------+
| 2020-01-09 14:38:48 |
+---------------------+
1 row in set (0,00 sec)
*/
```

```sql

INSERT INTO `posts` 
  (`title`, `content`, `thematic`, `lesson`,  `status`, `published_at`, `user_id`)
VALUES
("Géométrie dans l'espace chapitre 1",  'blablabla', 'MATH', 'Algèbre Linéaire', 'published',NOW(),2 ),
("Géométrie dans l'espace chaptire 2",  'blablabla', 'MATH', 'algèbre Linéaire', 'published',NOW() + 3,2 ),
("Géométrie dans l'espace chapitre 3",  'blablabla', 'MATH', 'Algèbre linéaire', 'published',NOW() - 4,2 ),
('Algorithm Euclide', 'blablabla', 'ALGO', 'Recherche rapide', 'published', NOW()+1, 2);
```

Création des catégories :

```sql
INSERT INTO `categories` (`title`)
  VALUES
  ('Programming' ),
  ('Algorithm' ),
  ('Mathématiques'),
  ('Framework');
```

Maintenant nous allons créer les relations entre les catégories et les posts :

```sql
INSERT INTO category_post (`post_id`, `category_id`)
  VALUES
  (1,3 ),
  (2,3),
  (3,3),
  (4, 2);

```

### 1 Exercice 

Sélectionnez tous les posts dont l'intitulé de la leçon est "Algèbre Linéaire". Puis en utilisant BINARY('monIntitule') sélectionnez uniquement les leçons qui s'écrive(nt) exactement : "algèbre linéaire" en respectant la casse.

### 2 Exercice

Insérez maintenant les posts suivants dans une table que vous allez créer old_posts, vous les associerez à l'auteur alan@alan.fr :

```txt
- "Théorème de Pythagore", "Un super texte", "MATH", "Géométrie", "draft", "2009-02-09 11:55:51"
- "Quick sort", "Un super texte", "ALGO", "Algorithme rapide", "draft", "2010-02-09 11:55:51"
- "Laravel", "Un super texte", "FRAM", "Framework PHP", "draft", "2009-02-09 10:55:51"
- "Inversion de matrice", "Un super texte", "MATH", "Matrix inverse", "draft", "1998-11-09 10:55:51"
- "Bezout", "Un super texte", "MATH", "Arithmétique", "draft", "1998-02-09 11:55:51"

````

Nous voulons maintenant récupérez les anciens posts et les mettre dans la table posts, écrivez la commande qui vous permettra de les insérer rapidement. Notez que vous pouvez utiliser un SELECT dans un INSERT.

```sql

INSERT INTO maTable (name, ...)
SELECT (name, ...)

```

Pour finir mettez à jour les dates des anciens posts dans la table posts.

### 3 Exercice UPDATE

Ajoutez le champ time_to_write à la table posts ce champ a pour type TIME, il peut être NULL.

Une fois le champ ajouté à la table mettez à jour la table posts en ajoutant des temps de rédaction aléatoire. Attention au format de TIME, voyez ce qui suit :

- TIME : hh:mm:ss où respectivement hh est compris entre 0 et 12 et hh et ss entre 0 et 60 strictement.

Vous pouvez utiliser les commandes SQL suivante, pour voir comment elles marchent tapez les dans MySQL 

```sql

SELECT CONCAT(10, ':', 56 , ':', 20); 
/*10:56:20*/

SELECT ROUND( RAND() * 59) ;

```

Affichez tous les posts dans la thématique **MATH** qui ont été écrit en moins de 6h.

### 4 Exercice

Ajoutez maintenant un champ score à la table posts. C'est un décimal sur 3 chiffres, 2 avant la virgule et 1 après la virgule. Il peut être de type NULL.

Ajoutez des données d'exemple pour tous les posts.

In [2]:
import mysql.connector
from mysql.connector import errorcode